In [2]:
import numpy as np
import matplotlib.pyplot as plt
import alpha_recoil_sim as ar
import importlib, glob, os, shutil, subprocess

In [2]:
## first make a list of all the daughter isotopes we need
path = "/Users/dcmoore/Library/CloudStorage/GoogleDrive-david.c.moore@yale.edu/My Drive/yale/uspheres/alpha_recoils_Grimm/Sphere_Recoils_MC"

iso_list = ['Ac-225', 'Pb-212', 'At-211', 'Th-227', 'Ra-223']

iso_dict = {}

for iso in iso_list:
    ciso = iso[:2].lower()
    cA = iso[-3:]
    iso_dict[iso] = ar.parse_decay_chain(path + "/decay_data/" + ciso + "_" + cA + "_decay_chain.txt")

In [3]:
## go through the dictionary and make a list of all alpha decay daughters we need SRIM simulations for
daughter_list = []
for iso in iso_list:
    curr_dict = iso_dict[iso]
    
    curr_keys = curr_dict.keys()

    for k in curr_keys:
        if not '_daughters' in k: continue

        curr_daught = curr_dict[k]
        curr_alpha = curr_dict[k[:6] + "_decays"][:,1] > 0

        for j in range(len(curr_daught)):
            if( curr_alpha[j] and not curr_daught[j] in daughter_list):
                daughter_list.append(curr_daught[j]) 

print(daughter_list)


['Fr-221', 'At-217', 'Bi-213', 'Tl-209', 'Pb-209', 'Tl-208', 'Pb-208', 'Bi-207', 'Pb-207', 'Ra-223', 'Rn-219', 'Po-215', 'Pb-211', 'Tl-207']


In [15]:
## now generate the TRIM input files from the templates for the list of isotopes above
importlib.reload(ar)

material_list = ["SiO2", "Au", "Ag"]
recoil_energy = 200 ## keV
num_events = 10000 ## number of events for each isotope and material

fidx = 0
for mat in material_list:

    template_file = path + "/TRIM_input_files/TRIM.IN_%s.txt"%mat

    with open(template_file, 'r') as tf:
        template_lines = tf.readlines()

    for iso in daughter_list:

        output_file = path + "/TRIM_input_files/TRIM.IN_%d"%fidx

        with open(output_file, 'wt') as of:
            
            for lidx, l in enumerate(template_lines):

                if lidx == 0: ## write the first line as a header unchanged
                    of.write(template_lines[0])
                    continue

                ## update ion information
                if template_lines[lidx - 1].startswith("Ion:"):
                    Z, A = ar.get_Z_A_for_iso(iso)
                    ion_line = "    %d    %d    %d    0    %d    1    %d\n"%(Z, A, recoil_energy, num_events, num_events)
                    of.write(ion_line)
                elif template_lines[lidx - 1].startswith("Target material"):
                    new_start = "%s (%d) into "%(iso, recoil_energy)
                    newl = l[0] + new_start + l[14:]
                    of.write(newl)
                else:
                    of.write(l)

        fidx += 1


In [8]:
## code to run on windows computer to run TRIM jobs
input_files = glob.glob("C:/SRIM/Sphere_Recoils_MC/TRIM_input_files/*")

destination_file = r"C:/SRIM/TRIM.IN"
trim_path = r"C:\SRIM"
output_dir = r"C:\SRIM\output_files_batch"
exyz_file = r"C:\SRIM\SRIM Outputs\EXYZ.txt"

for file in input_files[:1]:
    if file.endswith('.txt'): continue

    file_idx = file.split("_")[-1]

    shutil.copy(file, destination_file)
    print("Copied %s to %s"%(file, destination_file)) 

    print(trim_path)
    os.chdir(trim_path)
    subprocess.call("./TRIM.exe")

    shutil.copy(exyz_file, output_dir + "\exyz_%s.txt"%file_idx)


Copied C:/SRIM/Sphere_Recoils_MC/TRIM_input_files\TRIM.IN_0 to C:/SRIM/TRIM.IN
C:\SRIM


TypeError: %d format: a number is required, not str